## TYTAN tutorial おすすめ10（線形回帰）

最終更新：2023年6月25日 by ビネクラ安田

取り組み方

*   Google Colabで取り組む場合：ファイル＞ドライブにコピーを保存　<font color="red">※このファイルを直接編集しても保存されません</font>
*   Jupyter Notebookに移す場合：ファイル＞ダウンロード＞.ipynbをダウンロード

参考リンク１

*   [TYTANチュートリアル一覧](https://github.com/tytansdk/tytan_tutorial)
*   [TYTANドキュメント](https://github.com/tytansdk/tytan/blob/main/document%20.md)

出展

*   [量子アニーリング（QUBO）で線形回帰（最小二乗法）](https://vigne-cla.com/21-28/)

### 問題
ビリリダマ10匹のデータがある。最小二乗法で回帰直線を求めよ。
```
height(m)	weight(kg)
0.31	5.75
0.4	8.56
0.47	8.42
0.4	7.78
0.54	10.25
0.36	6.79
0.56	11.51
0.43	7.66
0.32	6.99
0.6	10.61
```
<div align="center">
<img src="https://vigne-cla.com/wp-content/uploads/2023/06/21-28_1.png" width = 45%>
</div>

### このQUBO設定を使おう

<font color="red">量子ビットを8個（8bit）用いて0～255の整数を表す</font>

例）xを0～255の整数で表す（x0～x7を使用）
```
x = 128*x0 + 64*x1 + 32*x2 + 16*x3 + 8*x4 + 4*x5 + 2*x6 + x7
```

その他の条件式も気になる方は → [量子アニーリングのQUBOで設定可能な条件式まとめ（保存版）](https://vigne-cla.com/21-12/)

### 考え方

高さx、重さyとして「y=ax+b」の傾きaと切片bを求める。

参考に、Excelで回帰直線を求めたのがこちら。
<div align="center">
<img src="https://vigne-cla.com/wp-content/uploads/2023/06/21-28_2.png" width = 45%>
</div>

最小二乗法では**「誤差の二乗の合計」**ができるだけ小さくなるような傾きaと切片bを求める。そこで、量子アニーリングではこの**「誤差の二乗の合計」**をエネルギーの**ペナルティ**に設定してやれば、最小エネルギーの解を求めることがそのまま最小二乗法に相当する。

例えば x=0.31 に着目すると、実データは 5.75 で、近似直線上の値は (a×0.31 + b) である。誤差は (5.75 – (a×0.31 + b)) で、これの二乗をペナルティにする。

<div align="center">
<img src="https://vigne-cla.com/wp-content/uploads/2023/06/21-28_3.png" width = 40%>
</div>

ここで、aとbは量子ビットをそのまま割り当ててしまうと０と１しか取り得ない。そこで量子ビットを8bit使って0～255までの256段階の数を表す方法を活用する。この256段階を10～20に規格化することで**「10～20の間の256段階の数値」**を表せる。aは10～20の間と見積もってこのように表現する（10≦a＜20）。

```
#a0～a7は量子ビット（10≦a＜20）
a = 10 + 10 * ((128*a0 + 64*a1 + 32*a2 + 16*a3 + 8*a4 + 4*a5 + 2*a6 + a7) / 256)
```

同様に、0～1に規格化することで**「0～1の間の256段階の数値」**を表せる。bは0～1の間と見積もってこのようにする（0≦b＜1）。

```
#b0～b7は量子ビット（0≦b＜1）
b = 0 + 1 * ((128*b0 + 64*b1 + 32*b2 + 16*b3 + 8*b4 + 4*b5 + 2*b6 + b7) / 256)
```

誤差の二乗ペナルティはこうなる。

```
H += (5.75 - (a*0.31 + b))**2
```

このaとbはそれぞれ8個の量子ビットの集合体で、展開した式はかなり長くなる。「こんな式立てて大丈夫なの？」と心配になるが大丈夫。QUBOでは2個の量子ビットの掛け算まで設定可能で、3個以上の量子ビットの掛け算はあってはいけない。冷静に展開してみると3個以上の掛け算は現れないので大丈夫。

## コード

In [ ]:
!pip install git+https://github.com/tytansdk/tytan

In [ ]:
from tytan import *
import numpy as np

#量子ビットを用意する
a0 = symbols('a0')
a1 = symbols('a1')
a2 = symbols('a2')
a3 = symbols('a3')
a4 = symbols('a4')
a5 = symbols('a5')
a6 = symbols('a6')
a7 = symbols('a7')

b0 = symbols('b0')
b1 = symbols('b1')
b2 = symbols('b2')
b3 = symbols('b3')
b4 = symbols('b4')
b5 = symbols('b5')
b6 = symbols('b6')
b7 = symbols('b7')

#aを2進数（8bit）で表す、10～20で規格化
a = 10 + 10 * ((128*a0 + 64*a1 + 32*a2 + 16*a3 + 8*a4 + 4*a5 + 2*a6 + a7) / 256)
print(a)
#bを2進数（8bit）で表す、0～1で規格化
b = 0 + 1 * ((128*b0 + 64*b1 + 32*b2 + 16*b3 + 8*b4 + 4*b5 + 2*b6 + b7) / 256)

#データ1の誤差二乗ペナルティ
H = 0
H += (5.75 - (a*0.31 + b))**2

#以降同様
H += (8.56 - (a*0.4 + b))**2
H += (8.42 - (a*0.47 + b))**2
H += (7.78 - (a*0.4 + b))**2
H += (10.25 - (a*0.54 + b))**2
H += (6.79 - (a*0.36 + b))**2
H += (11.51 - (a*0.56 + b))**2
H += (7.66 - (a*0.43 + b))**2
H += (6.99 - (a*0.32 + b))**2
H += (10.61 - (a*0.6 + b))**2


#コンパイル
qubo, offset = Compile(H).get_qubo()
print(f'offset\n{offset}')

#サンプラー選択
solver = sampler.SASampler()

#サンプリング
result = solver.run(qubo)

#上位5件
for r in result[:5]:
    print(r)
    #2進数から戻して確認
    a0,a1,a2,a3,a4,a5,a6,a7 = list(r[0].values())[:8]
    b0,b1,b2,b3,b4,b5,b6,b7 = list(r[0].values())[8:]
    print(f'a = {10 + 10 * ((128*a0 + 64*a1 + 32*a2 + 16*a3 + 8*a4 + 4*a5 + 2*a6 + a7) / 256)}')
    print(f'b = {0 + 1 * ((128*b0 + 64*b1 + 32*b2 + 16*b3 + 8*b4 + 4*b5 + 2*b6 + b7) / 256)}')

5*a0 + 5*a1/2 + 5*a2/4 + 5*a3/8 + 5*a4/16 + 5*a5/32 + 5*a6/64 + 5*a7/128 + 10
offset
171.475400000000
[{'a0': 1, 'a1': 0, 'a2': 1, 'a3': 1, 'a4': 1, 'a5': 0, 'a6': 0, 'a7': 0, 'b0': 1, 'b1': 1, 'b2': 1, 'b3': 0, 'b4': 0, 'b5': 0, 'b6': 1, 'b7': 1}, -168.1485632324219, 27]
a = 17.1875
b = 0.88671875
[{'a0': 1, 'a1': 0, 'a2': 1, 'a3': 1, 'a4': 1, 'a5': 0, 'a6': 0, 'a7': 1, 'b0': 1, 'b1': 1, 'b2': 0, 'b3': 1, 'b4': 1, 'b5': 1, 'b6': 1, 'b7': 1}, -168.14852859497077, 23]
a = 17.2265625
b = 0.87109375
[{'a0': 1, 'a1': 0, 'a2': 1, 'a3': 1, 'a4': 0, 'a5': 1, 'a6': 1, 'a7': 1, 'b0': 1, 'b1': 1, 'b2': 1, 'b3': 0, 'b4': 0, 'b5': 1, 'b6': 1, 'b7': 1}, -168.14827102661138, 4]
a = 17.1484375
b = 0.90234375
[{'a0': 1, 'a1': 0, 'a2': 1, 'a3': 1, 'a4': 1, 'a5': 0, 'a6': 1, 'a7': 0, 'b0': 1, 'b1': 1, 'b2': 0, 'b3': 1, 'b4': 1, 'b5': 0, 'b6': 1, 'b7': 0}, -168.14825500488286, 4]
a = 17.265625
b = 0.8515625
[{'a0': 1, 'a1': 0, 'a2': 1, 'a3': 1, 'a4': 0, 'a5': 1, 'a6': 1, 'a7': 1, 'b0': 1, 'b1': 1, 'b2': 

最良の解は (a, b) = (17.188, 0.887) となった。Excelで求めた傾きと切片は (17.206, 0.879) だったためかなり近い値が得られたことになる。

aとbはそれぞれ8bitで表現したため256段階の飛び飛びの値しか取り得ない。したがって**厳密な最適パラメータは求まらない。**より厳密に近いパラメータを求めたければaは17～18の範囲に絞って、bは0.8～0.9の範囲に絞ってさらにアニーリングを繰り返すと良い。

一桁ずつ絞るのがシンプルだが、せっかちな人はもっと絞っても良い可能性がある。aのように10の範囲を8bitで刻んだ場合、刻み幅は`10/256≒0.04`である。したがって±0.02の範囲に厳密解があることが期待されるため＊、次回は17.168～17.208の範囲に絞って良い。

＊次数や係数のバランスによってはそうならないので過信は禁物

または、9bitで表現すれば512段階で刻めるのでそういうアプローチでも良い。

余談で、「ビリリダマは大きくなるほど密度が下がる」に興味があれば

→[ポケモンの高さと重さを調べてみたら何かおかしかった](https://vigne-cla.com/16-16/)

## おまけ：TYTANの便利関数を使用したコード

8bit表現などをサクッと設定できる関数が用意されているので紹介する。

「256段階を10～20に規格化する」の部分までやってくれるのでけっこう便利。

詳しくはドキュメント → [TYTANドキュメント](https://github.com/tytansdk/tytan/blob/main/document%20.md)

In [ ]:
from tytan import *
import numpy as np

#aを2進数（8bit）で表す、10～20で規格化
a = symbols_nbit(10, 20, 'a{}', num=8)
#bを2進数（8bit）で表す、0～1で規格化
b = symbols_nbit(0, 1, 'b{}', num=8)

#誤差二乗ペナルティ
H = 0
H += (5.75 - (a*0.31 + b))**2
H += (8.56 - (a*0.4 + b))**2
H += (8.42 - (a*0.47 + b))**2
H += (7.78 - (a*0.4 + b))**2
H += (10.25 - (a*0.54 + b))**2
H += (6.79 - (a*0.36 + b))**2
H += (11.51 - (a*0.56 + b))**2
H += (7.66 - (a*0.43 + b))**2
H += (6.99 - (a*0.32 + b))**2
H += (10.61 - (a*0.6 + b))**2


#コンパイル
qubo, offset = Compile(H).get_qubo()
print(f'offset\n{offset}')

#サンプラー選択
solver = sampler.SASampler()

#サンプリング
result = solver.run(qubo)

#上位5件
for r in result[:5]:
    print(r)
    print('a =', Auto_array(r[0]).get_nbit_value(a))
    print('b =', Auto_array(r[0]).get_nbit_value(b))

offset
741.515400000000
[{'a0': 1, 'a1': 1, 'a2': 0, 'a3': 1, 'a4': 1, 'a5': 1, 'a6': 0, 'a7': 0, 'b0': 1, 'b1': 1, 'b2': 1, 'b3': 0, 'b4': 0, 'b5': 0, 'b6': 1, 'b7': 1}, -738.1885632324219, 29]
a = 17.1875
b = 0.88671875
[{'a0': 1, 'a1': 1, 'a2': 0, 'a3': 1, 'a4': 1, 'a5': 1, 'a6': 0, 'a7': 1, 'b0': 1, 'b1': 1, 'b2': 0, 'b3': 1, 'b4': 1, 'b5': 0, 'b6': 1, 'b7': 0}, -738.1882550048826, 24]
a = 17.265625
b = 0.8515625
[{'a0': 1, 'a1': 1, 'a2': 0, 'a3': 1, 'a4': 1, 'a5': 0, 'a6': 1, 'a7': 1, 'b0': 1, 'b1': 1, 'b2': 1, 'b3': 0, 'b4': 1, 'b5': 1, 'b6': 0, 'b7': 0}, -738.1877349853514, 9]
a = 17.109375
b = 0.921875
[{'a0': 1, 'a1': 1, 'a2': 0, 'a3': 1, 'a4': 1, 'a5': 0, 'a6': 1, 'a7': 1, 'b0': 1, 'b1': 1, 'b2': 1, 'b3': 0, 'b4': 1, 'b5': 0, 'b6': 1, 'b7': 1}, -738.187651977539, 14]
a = 17.109375
b = 0.91796875
[{'a0': 1, 'a1': 1, 'a2': 1, 'a3': 0, 'a4': 0, 'a5': 0, 'a6': 0, 'a7': 0, 'b0': 1, 'b1': 1, 'b2': 0, 'b3': 0, 'b4': 0, 'b5': 0, 'b6': 0, 'b7': 0}, -738.180625, 3]
a = 17.5
b = 0.75
